Prediction with 1st month of each year

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:

##Importing libraries
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns

### Read Data with Memory optimization

In [4]:
def reduce_mem_usage():

    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    
    df=pd.read_csv('/content/drive/My Drive/PHD/train.csv',na_values=["NA","?",",",""])
    print("Data Type of columns in the data frame before optimization")
    print(df.dtypes)
    print()

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of the data frame before optimization is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of the data frame after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    print()
    
    print("Data type of columns in the data frame after optimization")
    print(df.dtypes)
    print()

    return df

#Optimi
train_data=reduce_mem_usage()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Data Type of columns in the data frame before optimization
date            object
locationId      object
item_id         object
unit_sales     float64
onpromotion     object
dtype: object

Memory usage of the data frame before optimization is 3879.12 MB
Memory usage of the data frame after optimization is: 970.02 MB
Decreased by 75.0%

Data type of columns in the data frame after optimization
date           category
locationId     category
item_id        category
unit_sales      float32
onpromotion    category
dtype: object



In [0]:
train_data.to_pickle("train_data_optimize2.pkl")

In [0]:
location_data = pd.read_csv('/content/drive/My Drive/PHD/location_details.csv')

Merge location data to train data to get type column

In [0]:
train_data.shape

(101688779, 5)

In [0]:
train_data=train_data.merge(location_data,how='left',left_on='locationId', right_on='location_id')  #doing left join bcz item_data does not have time column and time column is very imp here

In [9]:
train_data.shape

(101688779, 10)

In [10]:
train_data.columns

Index(['date', 'locationId', 'item_id', 'unit_sales', 'onpromotion',
       'location_id', 'city', 'state', 'type', 'cluster'],
      dtype='object')

In [0]:
train_data.drop(columns= [ 'location_id', 'city', 'state','cluster' ], axis=1, inplace=True)

In [13]:
import gc
gc.collect()

0

seperate date in to day month year

In [0]:
import datetime

In [0]:
train_data['date']=pd.to_datetime(train_data['date'],format='%Y-%m-%d')

In [0]:
train_data['month']=train_data['date'].dt.month

In [0]:
train_data['year']=train_data['date'].dt.year

In [0]:
train_data['day']=train_data['date'].dt.day

In [19]:
train_data.shape

(101688779, 9)

In [20]:
train_data.head()

,date,locationId,item_id,unit_sales,onpromotion,type,month,year,day
0,2015-01-01,location_25,item_103665,7.0,NaN,D,1,2015,1
1,2015-01-01,location_25,item_105574,1.0,NaN,D,1,2015,1
2,2015-01-01,location_25,item_105575,2.0,NaN,D,1,2015,1
3,2015-01-01,location_25,item_108079,1.0,NaN,D,1,2015,1
4,2015-01-01,location_25,item_108701,1.0,NaN,D,1,2015,1


#### The test data is read and the operations of training data are repeated on the same 

In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [24]:
test_data = pd.read_csv('/content/drive/My Drive/PHD/test.csv',na_values=["NA","?",",",""])
test_data.head()

,id,date,locationId,item_id,onpromotion
0,0,2019-01-01,location_25,item_99197,False
1,1,2019-01-01,location_25,item_103665,False
2,2,2019-01-01,location_25,item_105574,False
3,3,2019-01-01,location_25,item_105857,False
4,4,2019-01-01,location_25,item_106716,False


During trying different models type came as important feature

In [0]:
test_data = pd.merge(test_data,location_data,left_on='locationId', right_on = 'location_id', how = 'left')

In [27]:
test_data.head()

,id,date,locationId,item_id,onpromotion,type
0,0,2019-01-01,location_25,item_99197,False,D
1,1,2019-01-01,location_25,item_103665,False,D
2,2,2019-01-01,location_25,item_105574,False,D
3,3,2019-01-01,location_25,item_105857,False,D
4,4,2019-01-01,location_25,item_106716,False,D


In [0]:
test_data.drop(columns= [ 'location_id', 'city', 'state','cluster' ], axis=1, inplace=True)

In [28]:
import gc
gc.collect()

257

In [0]:
test_data['date']=pd.to_datetime(test_data['date'],format='%Y-%m-%d')

In [0]:
test_data['month']=test_data['date'].dt.month

In [0]:
test_data['year']=test_data['date'].dt.year

In [0]:
test_data['day']=test_data['date'].dt.day

In [33]:
test_data.shape

(1482295, 9)

In [0]:
test_data.head()

,id,date,locationId,item_id,onpromotion,type,month,year,day
0,0,2019-01-01,location_25,item_99197,False,D,1,2019,1
1,1,2019-01-01,location_25,item_103665,False,D,1,2019,1
2,2,2019-01-01,location_25,item_105574,False,D,1,2019,1
3,3,2019-01-01,location_25,item_105857,False,D,1,2019,1
4,4,2019-01-01,location_25,item_106716,False,D,1,2019,1


Taking january data of each year which gave better results

In [0]:
train_data = train_data[train_data['month'] == 1]

convert the date number into day of the week. 


In [0]:
train_data['date'] = pd.to_datetime(train_data['date'])

In [0]:
train_data['dow'] = train_data['date'].dt.dayofweek

In [0]:
test_data['date'] = pd.to_datetime(test_data['date'])

In [0]:
test_data['dow'] = test_data['date'].dt.dayofweek

In [0]:
train_data.shape

(7726971, 10)

In [0]:
test_data.shape

(1482295, 10)

In [0]:
print(train_data.head())
print(test_data.head())

        date   locationId      item_id  unit_sales  ... month  year  day  dow
0 2015-01-01  location_25  item_103665         7.0  ...     1  2015    1    3
1 2015-01-01  location_25  item_105574         1.0  ...     1  2015    1    3
2 2015-01-01  location_25  item_105575         2.0  ...     1  2015    1    3
3 2015-01-01  location_25  item_108079         1.0  ...     1  2015    1    3
4 2015-01-01  location_25  item_108701         1.0  ...     1  2015    1    3

[5 rows x 10 columns]
   id       date   locationId      item_id  ...  month  year  day  dow
0   0 2019-01-01  location_25   item_99197  ...      1  2019    1    1
1   1 2019-01-01  location_25  item_103665  ...      1  2019    1    1
2   2 2019-01-01  location_25  item_105574  ...      1  2019    1    1
3   3 2019-01-01  location_25  item_105857  ...      1  2019    1    1
4   4 2019-01-01  location_25  item_106716  ...      1  2019    1    1

[5 rows x 10 columns]


In [0]:
train_data.tail()

,locationId,item_id,unit_sales,onpromotion,type,dow
69282588,location_54,item_2026801,4.0,0,C,2
69282589,location_54,item_2026893,1.0,0,C,2
69282590,location_54,item_2026983,1.0,0,C,2
69282591,location_54,item_2027090,1.0,0,C,2
69282592,location_54,item_2027252,3.0,0,C,2


Drop date year day month

In [0]:
train_data.drop(columns= ['date', 'year', 'month', 'day'], axis=1, inplace=True)

In [0]:
test_data.drop(columns = ['id', 'date','year', 'month', 'day' ], axis = 1, inplace = True)

In [0]:
train_data.dtypes

locationId     category
item_id        category
unit_sales      float32
onpromotion    category
type           category
dow               int64
dtype: object

In [0]:
train_data[["locationId","type"]]=train_data[["locationId","type"]].astype("category")

In [43]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7726971 entries, 0 to 69282592
Data columns (total 6 columns):
locationId     category
item_id        category
unit_sales     float32
onpromotion    category
type           category
dow            int64
dtypes: category(4), float32(1), int64(1)
memory usage: 184.4 MB


In [0]:
test_data.dtypes

locationId     category
item_id        category
onpromotion    category
type           category
dow               int64
dtype: object

In [0]:
test_data[["locationId","item_id","onpromotion",'type']]=test_data[["locationId","item_id","onpromotion",'type']].astype("category")

Fill NA

In [0]:
train_data['onpromotion'] = train_data['onpromotion'].fillna(0)

In [0]:
train_data['onpromotion'] = train_data['onpromotion'].astype('int').astype('category')

In [0]:
test_data['onpromotion'] = test_data['onpromotion'].fillna(0)

In [0]:
test_data['onpromotion'] = test_data['onpromotion'].astype('int').astype('category')

In [0]:
train_data.dtypes

locationId     category
item_id        category
unit_sales      float32
onpromotion    category
type           category
dow               int64
dtype: object

In [0]:
test_data.dtypes

locationId     category
item_id        category
onpromotion    category
type           category
dow               int64
dtype: object

In [0]:
train_data.isnull().sum()

locationId     0
item_id        0
unit_sales     0
onpromotion    0
type           0
dow            0
dtype: int64

In [0]:
test_data.isnull().sum()

locationId     0
item_id        0
onpromotion    0
type           0
dow            0
dtype: int64

In [0]:
test_data.head()

,location_id,item_id,onpromotion,type,dow
0,location_25,item_99197,0,D,1
1,location_25,item_103665,0,D,1
2,location_25,item_105574,0,D,1
3,location_25,item_105857,0,D,1
4,location_25,item_106716,0,D,1


In [0]:
train_data.to_pickle("train_data_optimize3.pkl")

In [0]:

test_data.to_pickle("test_data_optimize.pkl")

drop negative values

In [0]:
train_data = train_data[~(train_data['unit_sales'] < 0)]

X train and y train

In [0]:
y=train_data["unit_sales"]
X=train_data.drop('unit_sales', axis=1)

In [0]:
X.dtypes

locationId     category
item_id        category
onpromotion    category
type           category
dow               int64
dtype: object

In [0]:
test_data.dtypes

locationId     category
item_id        category
onpromotion    category
type           category
dow               int64
dtype: object

In [0]:
min(y)

0.004999999888241291

In [0]:
X.head()

,locationId,item_id,onpromotion,type,dow
0,location_25,item_103665,0,D,3
1,location_25,item_105574,0,D,3
2,location_25,item_105575,0,D,3
3,location_25,item_108079,0,D,3
4,location_25,item_108701,0,D,3


In [0]:
test_data.head()

,locationId,item_id,onpromotion,type,dow
0,location_25,item_99197,0,D,1
1,location_25,item_103665,0,D,1
2,location_25,item_105574,0,D,1
3,location_25,item_105857,0,D,1
4,location_25,item_106716,0,D,1


Categorical to numerical

In [0]:
from sklearn.preprocessing import OneHotEncoder

In [0]:
enc = OneHotEncoder(handle_unknown='ignore')

In [55]:
%%time
X = enc.fit_transform(X)

CPU times: user 7.24 s, sys: 214 ms, total: 7.45 s
Wall time: 7.46 s


In [56]:
X.shape

(7726439, 3535)

In [0]:
test_data = enc.transform(test_data)

### Train - Test Split 


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state = 5)   

### Final Model

In [0]:
from xgboost import XGBRegressor

In [80]:
%%time
xgb = XGBRegressor(n_estimators=11, objective='reg:linear', eta = 0.5, booster='gbtree',
                   max_depth = 10, subsample= 0.8, colsample_bytree= 0.7, silent= 1, min_child_weight=5,
                   gamma=0.5, learning_rate=0.01)

CPU times: user 26 µs, sys: 20 µs, total: 46 µs
Wall time: 51.7 µs


In [81]:
%%time 
xgb = xgb.fit(X_train, y_train)

CPU times: user 1min 55s, sys: 552 ms, total: 1min 56s
Wall time: 1min 57s


In [0]:
y_val_pred_xgb = xgb.predict(X_val)

In [0]:
y_train_pred_xgb = xgb.predict(X_train)

In [0]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error

In [85]:
print(mae(y_val, y_val_pred_xgb))
print(mae( y_train, y_train_pred_xgb))

7.5491304
7.5542984


In [86]:
print((np.mean((np.abs(y_train-y_train_pred_xgb))/y_train))*100)

64.30268883705139


#### Prediction on test data

In [0]:
test_pred = xgb.predict(test_data)

In [89]:
test_pred.shape

(1482295,)

In [0]:
test_pred1=pd.read_csv('/content/drive/My Drive/PHD/test.csv',na_values=["NA","?",",",""])

In [0]:
test_pred1['unit_sales']=test_pred

In [92]:
test_pred1.columns

Index(['id', 'date', 'locationId', 'item_id', 'onpromotion', 'unit_sales'], dtype='object')

In [0]:
test_pred1.drop(columns= ['date', 'locationId', 'item_id', 'onpromotion'], axis=1, inplace=True)

In [94]:
test_pred1.head()

,id,unit_sales
0,0,1.20847
1,1,1.20847
2,2,1.20847
3,3,1.20847
4,4,1.20847


In [95]:
test_pred1.shape

(1482295, 2)

In [0]:
test_pred1.to_csv(r'pred_xgb.csv',index=False)

XGB gave 62.65 MAPE

Business answers

Business units belonging to which cluster will see the highest amount of  sales for the first 15 days of 2019?  

In [0]:
pred_xgb = pd.read_csv('/content/drive/My Drive/PHD/pred_xgb.csv')

In [113]:
pred_xgb.head()

,id,unit_sales
0,0,1.20847
1,1,1.20847
2,2,1.20847
3,3,1.20847
4,4,1.20847


In [0]:
pred_xgb.drop('id',axis=1,inplace=True)

In [115]:
pred_xgb.shape

(1482295, 1)

In [0]:
test_data= pd.read_csv('/content/drive/My Drive/PHD/test.csv')

In [119]:
test_data.head()

,id,date,locationId,item_id,onpromotion
0,0,2019-01-01,location_25,item_99197,False
1,1,2019-01-01,location_25,item_103665,False
2,2,2019-01-01,location_25,item_105574,False
3,3,2019-01-01,location_25,item_105857,False
4,4,2019-01-01,location_25,item_106716,False


In [0]:
item_data = pd.read_csv('/content/drive/My Drive/PHD/item_details.csv')

Merge test_data with item and location data

In [0]:
test_data=test_data.merge(location_data,how='left',left_on='locationId', right_on='location_id')  #doing left join bcz item_data does not have time column and time column is very imp here

In [0]:
test_data=test_data.merge(item_data,how='left',left_on='item_id', right_on='item_id')  #doing left join bcz item_data does not have time column and time column is very imp here

In [125]:
test_data.head()

,id,date,locationId,item_id,onpromotion,location_id,city,state,type,cluster,category_of_item,class,perishable,unit_sales
0,0,2019-01-01,location_25,item_99197,False,location_25,Kanpur,Uttar Pradesh,D,1,grocery_items,class_1067,0,1.20847
1,1,2019-01-01,location_25,item_103665,False,location_25,Kanpur,Uttar Pradesh,D,1,baked_items / bread_based,class_2712,1,1.20847
2,2,2019-01-01,location_25,item_105574,False,location_25,Kanpur,Uttar Pradesh,D,1,grocery_items,class_1045,0,1.20847
3,3,2019-01-01,location_25,item_105857,False,location_25,Kanpur,Uttar Pradesh,D,1,grocery_items,class_1092,0,1.20847
4,4,2019-01-01,location_25,item_106716,False,location_25,Kanpur,Uttar Pradesh,D,1,grocery_items,class_1032,0,1.20847


In [126]:
test_data.shape

(1482295, 14)

In [0]:
test_data['unit_sales']=pred_xgb

In [0]:
cluster_group=test_data.groupby('cluster').unit_sales.sum().reset_index()

In [129]:
cluster_group

,cluster,unit_sales
0,1,107795.394090
1,2,76616.365812
2,3,160355.432013
3,4,108057.404685
4,5,74664.968902
5,6,212187.738781
6,7,51062.027015
7,8,158774.421300
8,9,70740.300811
9,10,186358.650548


In [130]:
cluster_group[cluster_group.unit_sales==cluster_group.unit_sales.max()]

,cluster,unit_sales
13,14,263489.392676


Cluster 14 is the answer

 What are the top 10 selling items in this cluster?  

In [0]:
cluster14=test_data[test_data.cluster==14]

In [133]:
cluster14.head()

,id,date,locationId,item_id,onpromotion,location_id,city,state,type,cluster,category_of_item,class,perishable,unit_sales
95269,95269,2019-01-02,location_46,item_99197,False,location_46,Bengaluru,Karnataka,A,14,grocery_items,class_1067,0,1.68769
95270,95270,2019-01-02,location_46,item_103520,False,location_46,Bengaluru,Karnataka,A,14,grocery_items,class_1028,0,1.68769
95271,95271,2019-01-02,location_46,item_103665,False,location_46,Bengaluru,Karnataka,A,14,baked_items / bread_based,class_2712,1,1.68769
95272,95272,2019-01-02,location_46,item_105574,False,location_46,Bengaluru,Karnataka,A,14,grocery_items,class_1045,0,1.68769
95273,95273,2019-01-02,location_46,item_105575,False,location_46,Bengaluru,Karnataka,A,14,grocery_items,class_1045,0,1.68769


In [0]:
cluster14_sales=cluster14.groupby('item_id').unit_sales.sum().reset_index()

In [135]:
cluster14_sales

,item_id,unit_sales
0,item_1000866,33.309711
1,item_1001305,74.228811
2,item_1003679,94.757450
3,item_1004545,33.171534
4,item_1004550,94.757450
...,...,...
3446,item_996606,93.069760
3447,item_996613,55.664217
3448,item_999545,94.757450
3449,item_999546,94.757450


In [0]:
sorted_cluster14=cluster14_sales.sort_values(by='unit_sales',ascending=False)

In [0]:
top_10=sorted_cluster14.iloc[0:10,:]

In [138]:
top_10

,item_id,unit_sales
1356,item_1503844,987.135486
1168,item_1463992,938.627007
2887,item_584028,888.615738
1293,item_1473474,830.918983
3117,item_807493,766.592867
63,item_1047679,514.159879
1169,item_1463993,507.320218
1533,item_1695835,475.650851
949,item_1430040,461.222664
280,item_1143685,375.502315


top 10 sales item wise

### What is the rate of purchase per week 


In [139]:
(top_10['unit_sales']/15)*7

1356    460.663227
1168    438.025936
2887    414.687344
1293    387.762192
3117    357.743338
63      239.941277
1169    236.749435
1533    221.970397
949     215.237243
280     175.234414
Name: unit_sales, dtype: float64

In [0]:
top_10['rate_of_purchase_per_week']=(top_10['unit_sales']/15)*7    #unit sales here is the sum of 15 days sale item wise..so to find per week..first chnge it to per day by didviding it by 15..then make per week by multiplying by 7

In [141]:
top_10

,item_id,unit_sales,rate_of_purchase_per_week
1356,item_1503844,987.135486,460.663227
1168,item_1463992,938.627007,438.025936
2887,item_584028,888.615738,414.687344
1293,item_1473474,830.918983,387.762192
3117,item_807493,766.592867,357.743338
63,item_1047679,514.159879,239.941277
1169,item_1463993,507.320218,236.749435
1533,item_1695835,475.650851,221.970397
949,item_1430040,461.222664,215.237243
280,item_1143685,375.502315,175.234414


rate of purchase item wise